In [1]:
!pip install -q datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [3]:
!unzip *.zip > /dev/null

In [6]:
import re
from datasets import load_from_disk

In [5]:
dataset = load_from_disk('staqc_man_python_codegen.hf')
print(dataset)
print(dataset['train'][0]['text'])

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 3694
    })
    test: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 410
    })
})
offset = 0
data1 = np.memmap('tmp', dtype='i', mode='r+', order='F',
                  offset=0, shape=(size1))
offset += size1*byte_size
data2 = np.memmap('tmp', dtype='i', mode='r+', order='F',
                  offset=offset, shape=(size2))
offset += size1*byte_size
data3 = np.memmap('tmp', dtype='i', mode='r+', order='F',
                  offset=offset, shape=(size3))



In [22]:
comments_regex = r'#.*'
notcomments_regex = r'^([^#]*)(#.*)?$'

def extract(code):
    comments = [c.strip() for c in re.findall(comments_regex, code, re.MULTILINE)]
    src = [c[0].strip(' \n\t#') for c in re.findall(notcomments_regex, code, re.MULTILINE)]
    comments = [s for s in comments if len(s)]
    src = [s for s in src if len(s)]

    return src, comments

code = """
def hello():
    # Say hello
    print('Hello') # Comment

# Call it
hello()
"""

src, comments = extract(code)
print(src)
print(comments)

['def hello():', "print('Hello')", 'hello()']
['# Say hello', '# Comment', '# Call it']


## Make datasets

In [24]:
def extract_comments(e):
    src, comments = extract(e['text'])
    e['text'] = '\n'.join(comments)
    return e

comments_dataset = dataset.map(extract_comments).filter(lambda e: len(e['text']) > 0)
print(comments_dataset['test'][0]['text'])

Filter:   0%|          | 0/3694 [00:00<?, ? examples/s]

Filter:   0%|          | 0/410 [00:00<?, ? examples/s]

# last value returned by decorated function is accessible as 'last' attribute
# w is the 'wrapper' function


In [26]:
def extract_src(e):
    src, comments = extract(e['text'])
    e['text'] = '\n'.join(src)
    return e

src_dataset = dataset.map(extract_src).filter(lambda e: len(e['text']) > 0)
print(src_dataset['test'][0]['text'])

Map:   0%|          | 0/3694 [00:00<?, ? examples/s]

Map:   0%|          | 0/410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3694 [00:00<?, ? examples/s]

Filter:   0%|          | 0/410 [00:00<?, ? examples/s]

class SpecificUserProfile(UserProfile):
    referrer = models.ForeignKey('self', related_name='referred')


In [29]:
print(comments_dataset)
print(src_dataset)

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 2008
    })
    test: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 77
    })
})
DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 3494
    })
    test: Dataset({
        features: ['question_id', 'question', 'text', 'label'],
        num_rows: 408
    })
})


## Export

In [27]:
!rm -rf staqc_man_python_codegen_comments.hf
comments_dataset.save_to_disk("staqc_man_python_codegen_comments.hf")
!rm -f staqc_man_python_codegen_comments.hf.zip
!zip -r staqc_man_python_codegen_comments.hf.zip staqc_man_python_codegen_comments.hf

Saving the dataset (0/1 shards):   0%|          | 0/2008 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/77 [00:00<?, ? examples/s]

  adding: staqc_man_python_codegen_comments.hf/ (stored 0%)
  adding: staqc_man_python_codegen_comments.hf/dataset_dict.json (stored 0%)
  adding: staqc_man_python_codegen_comments.hf/test/ (stored 0%)
  adding: staqc_man_python_codegen_comments.hf/test/dataset_info.json (deflated 54%)
  adding: staqc_man_python_codegen_comments.hf/test/data-00000-of-00001.arrow (deflated 54%)
  adding: staqc_man_python_codegen_comments.hf/test/state.json (deflated 42%)
  adding: staqc_man_python_codegen_comments.hf/train/ (stored 0%)
  adding: staqc_man_python_codegen_comments.hf/train/dataset_info.json (deflated 54%)
  adding: staqc_man_python_codegen_comments.hf/train/data-00000-of-00001.arrow (deflated 63%)
  adding: staqc_man_python_codegen_comments.hf/train/state.json (deflated 42%)


In [28]:
!rm -rf staqc_man_python_codegen_source.hf
src_dataset.save_to_disk("staqc_man_python_codegen_source.hf")
!rm -f staqc_man_python_codegen_source.hf.zip
!zip -r staqc_man_python_codegen_source.hf.zip staqc_man_python_codegen_source.hf

Saving the dataset (0/1 shards):   0%|          | 0/3494 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/408 [00:00<?, ? examples/s]

  adding: staqc_man_python_codegen_source.hf/ (stored 0%)
  adding: staqc_man_python_codegen_source.hf/dataset_dict.json (stored 0%)
  adding: staqc_man_python_codegen_source.hf/test/ (stored 0%)
  adding: staqc_man_python_codegen_source.hf/test/dataset_info.json (deflated 54%)
  adding: staqc_man_python_codegen_source.hf/test/data-00000-of-00001.arrow (deflated 61%)
  adding: staqc_man_python_codegen_source.hf/test/state.json (deflated 42%)
  adding: staqc_man_python_codegen_source.hf/train/ (stored 0%)
  adding: staqc_man_python_codegen_source.hf/train/dataset_info.json (deflated 54%)
  adding: staqc_man_python_codegen_source.hf/train/data-00000-of-00001.arrow (deflated 64%)
  adding: staqc_man_python_codegen_source.hf/train/state.json (deflated 42%)
